In [1]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")

MODEL_NAME = "mixtral-8x7b-32768"

llm = ChatGroq(
    temperature=0,
    model_name=MODEL_NAME,
    groq_api_key=groq_api_key
)

In [9]:
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a cpecified city
    
    Args:
        city (str): The name of the city where you want to check the weather
        
    Returns:
        str: A description of the current weather in a specified city
    """
    return "Sunny, 33°C"


tools = [multiply, add, fake_weather_api]

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant skilled in math operations."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [11]:
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
result = agent_executor.invoke({"input": "What's 3 plus 5 multiplied by 2?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 5, 'y': 2}`


10.0
Invoking: `add` with `{'x': 3, 'y': 10.0}`


13.0The final answer to the user's question is: 13.0.

> Finished chain.
{'input': "What's 3 plus 5 multiplied by 2?", 'output': "The final answer to the user's question is: 13.0."}


In [12]:
result = agent_executor.invoke({"input": "What's the current weather in Tel-Aviv?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `fake_weather_api` with `{'city': 'Tel-Aviv'}`


Sunny, 33°CThe current weather in Tel-Aviv is Sunny, 33°C.

> Finished chain.
{'input': "What's the current weather in Tel-Aviv?", 'output': 'The current weather in Tel-Aviv is Sunny, 33°C.'}


In [13]:
result

{'input': "What's the current weather in Tel-Aviv?",
 'output': 'The current weather in Tel-Aviv is Sunny, 33°C.'}